# Predicted SSVEP Stimulus Frequency using CCA

### Importing libraries

In [1]:
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from sklearn.cross_decomposition import CCA

### Getting Data

In [3]:
mat_contents = sio.loadmat('Data/data/s1.mat')

In [4]:
eeg_data = mat_contents["eeg"]
chan_locs = ['PO7', 'PO3', 'POz', 'PO4', 'PO8', 'O1', 'Oz', 'O2']
eeg_data = mat_contents['eeg']
# 1st dim: 12   -- target
# 2nd dim: 8    -- channels
# 3rd dim: 1114 -- timepoints
# 4th dim: 15   -- trials

### Setting up the filters

In [5]:
def butter_highpass_filter(data, cutoff, nyq, order=5):
    """Butterworth high-pass filter.
    Args:
        data (array_like): data to be filtered.
        cutoff (float): cutoff frequency.
        order (int): order of the filter.
    Returns:
        array: filtered data."""
    normal_cutoff = cutoff / nyq  # normalized cutoff frequency
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    filtered_data = signal.filtfilt(b, a, data)
    return filtered_data

def butter_lowpass_filter(data, cutoff, nyq, order=5):
    """Butterworth low-pass filter.
    Args:
        data (array_like): data to be filtered.
        cutoff (float): cutoff frequency.
        order (int): order of the filter.
    Returns:
        array: filtered data."""
    normal_cutoff = cutoff / nyq  # normalized cutoff frequency
    b, a = signal.butter(order, normal_cutoff, btype='low', analog=False)
    filtered_data = signal.lfilter(b, a, data)
    return filtered_data

# Filter parameters
fps = 256  # sampling frequency
cutoff_high = 6  # cutoff frequency of the high-pass filter
cutoff_low = 50  # cutoff frequency of the low-pass filter
nyq = 0.5 * fps  # Nyquist frequency (half of the sampling frequency).  It represents the highest frequency that can be accurately represented in a discrete-time signal.

# Filter Dataset

In [20]:
# Doing just 9 Hz, 11 Hz, 13 Hz, 14 Hz
# Doing just Oz electrode (index 6)

filtered_epochs = [] # Will contain the all epochs of data
target = [] # Will contain the target stimulus frequency to the corresponding filtered_epoch item
electrode_id = 6 # To only get Oz


labels = [9.25, 11.25, 13.25, 14.25]
stimulus_id = [0, 1, 2, 8]

# Four classes
for i in range(4):

    # Cycle each through trial in the class
    for j in range(15):

        # Temporary array to hold epoch (For 3rd dimension, we segment from the 39th timepoint and onwards since 
        # the 39th timepoint is the onset of the stimulus )
        temp_epoch = np.array(eeg_data[stimulus_id[i], electrode_id, 38:, j]).flatten()

        # apply the band-pass filter
        temp_epoch = butter_highpass_filter(
            data=temp_epoch,
            cutoff=cutoff_high,
            nyq=nyq,
            order=4)

        temp_epoch = butter_lowpass_filter(
            data=temp_epoch,
            cutoff=cutoff_low,
            nyq=nyq,
            order=4)

        # Append the epoch data and target label
        filtered_epochs.append(temp_epoch)
        target.append(labels[i])

In [21]:
# Function to get the index of the frequency from the FFT
def closest(lst, K):
    return (np.abs(lst - K)).argmin()

In [22]:
# Function to get the FFT values
def GetFFT(data_):
    
    data_len = len(data_)  # number of observations
    fourier_transform = np.fft.fft(data_)  # compute FFT
    fourier_transform = fourier_transform / data_len  # normalize values
    fourier_transform = fourier_transform[:int(data_len/2)]  # take half of the data

    time_period = data_len / fps  # time period of the signal
    values = np.arange(int(data_len/2))  # x-axis values up to Nyquist frequency
    frequencies = values / time_period  # x-axis values in Hz

    return frequencies, abs(fourier_transform)

In [23]:
# Holds accuracy
acc = 0

# Loops over all epochs
for i in range(len(filtered_epochs)):

    # Get the FFT calculations of the epoch
    freq, power = GetFFT(filtered_epochs[i])

    # Hold the features of the epoch (4 features corresponding to the target labels)
    features = []
    for j in labels: # Loop 4 times over labels (9.25, 11.25, 13.25, 14.25)
        ind = closest(freq, j) # Get the index of the target frequency from the fft
        val = abs(power)[ind] # Get the power of the target frequency
        features.append(val) 

    print(f"Actual Freq: {target[i]}, Predicted Freq: {labels[np.argmax(features)]}")

    # Increase accuracy if target was successfully found
    if (target[i] == labels[np.argmax(features)]):
        acc += 1

acc = acc/len(filtered_epochs)
print(acc)

Actual Freq: 9.25, Predicted Freq: 9.25
Actual Freq: 9.25, Predicted Freq: 11.25
Actual Freq: 9.25, Predicted Freq: 11.25
Actual Freq: 9.25, Predicted Freq: 9.25
Actual Freq: 9.25, Predicted Freq: 11.25
Actual Freq: 9.25, Predicted Freq: 9.25
Actual Freq: 9.25, Predicted Freq: 13.25
Actual Freq: 9.25, Predicted Freq: 14.25
Actual Freq: 9.25, Predicted Freq: 9.25
Actual Freq: 9.25, Predicted Freq: 9.25
Actual Freq: 9.25, Predicted Freq: 13.25
Actual Freq: 9.25, Predicted Freq: 9.25
Actual Freq: 9.25, Predicted Freq: 9.25
Actual Freq: 9.25, Predicted Freq: 9.25
Actual Freq: 9.25, Predicted Freq: 9.25
Actual Freq: 11.25, Predicted Freq: 9.25
Actual Freq: 11.25, Predicted Freq: 9.25
Actual Freq: 11.25, Predicted Freq: 11.25
Actual Freq: 11.25, Predicted Freq: 11.25
Actual Freq: 11.25, Predicted Freq: 11.25
Actual Freq: 11.25, Predicted Freq: 11.25
Actual Freq: 11.25, Predicted Freq: 11.25
Actual Freq: 11.25, Predicted Freq: 11.25
Actual Freq: 11.25, Predicted Freq: 11.25
Actual Freq: 11.25